<a href="https://colab.research.google.com/github/channeladam/MOOCs/blob/master/SchoolOfAI/DataLit/Week03/8-Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataLit Week 3 Homework Assignment

## Question
Is there a correlation between solar flare activity and rates of suicide?


## Data Sources

The data was sourced from the following Kaggle datasets.

* Suicide Rates Overview 1985 to 2016
    - https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

* Solar Flares 2002 - 2016
    - https://www.kaggle.com/khsamaha/solar-flares-rhessi



# Imports

In [0]:
import pandas as pd
import numpy as np
import scipy as sp

from sklearn import preprocessing

from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, FixedTicker, SingleIntervalTicker, NumeralTickFormatter, Legend, HoverTool

from google.colab import files

# Data Preparation

## Suicide Data

In [232]:
!wget -N "https://raw.githubusercontent.com/channeladam/MOOCs/master/SchoolOfAI/DataLit/Week03/Data/suicide-rates-overview-1985-to-2016.csv"

--2019-02-17 13:58:15--  https://raw.githubusercontent.com/channeladam/MOOCs/master/SchoolOfAI/DataLit/Week03/Data/suicide-rates-overview-1985-to-2016.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2706377 (2.6M) [text/plain]
Saving to: ‘suicide-rates-overview-1985-to-2016.csv’

suicide-rates-overv 100%[===================>]   2.58M  --.-KB/s    in 0.06s   

Last-modified header missing -- time-stamps turned off.
2019-02-17 13:58:15 (46.4 MB/s) - ‘suicide-rates-overview-1985-to-2016.csv’ saved [2706377/2706377]



In [205]:
!ls
!pwd

hessi.solar.flare.2002to2016.csv
sample_data
suicide-rates-overview-1985-to-2016.csv
suicide-rates-overview-1985-to-2016.csv.1
suicide-rates-overview-1985-to-2016.csv.2
/content


In [0]:
suicide = pd.read_csv('/content/suicide-rates-overview-1985-to-2016.csv')

In [207]:
suicide.head(10)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers
5,Albania,1987,female,75+ years,1,35600,2.81,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
6,Albania,1987,female,35-54 years,6,278800,2.15,Albania1987,NaN,"2,156,624,900",796,Silent
7,Albania,1987,female,25-34 years,4,257200,1.56,Albania1987,NaN,"2,156,624,900",796,Boomers
8,Albania,1987,male,55-74 years,1,137500,0.73,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
9,Albania,1987,female,5-14 years,0,311000,0.00,Albania1987,NaN,"2,156,624,900",796,Generation X


In [208]:
suicide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
country               27820 non-null object
year                  27820 non-null int64
sex                   27820 non-null object
age                   27820 non-null object
suicides_no           27820 non-null int64
population            27820 non-null int64
suicides/100k pop     27820 non-null float64
country-year          27820 non-null object
HDI for year          8364 non-null float64
 gdp_for_year ($)     27820 non-null object
gdp_per_capita ($)    27820 non-null int64
generation            27820 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


In [209]:
suicide = suicide.drop(['HDI for year'], axis=1)

# Remove all suicide data prior to 2002 since we only have solar flare activity from 2002
suicide = suicide.loc[ suicide.year >= 2002 ].reset_index()

suicide.head(10)

,index,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),generation
0,156,Albania,2002,male,75+ years,4,31007,12.90,Albania2002,"4,435,078,648",1573,Silent
1,157,Albania,2002,male,25-34 years,23,206286,11.15,Albania2002,"4,435,078,648",1573,Generation X
2,158,Albania,2002,male,35-54 years,35,382139,9.16,Albania2002,"4,435,078,648",1573,Boomers
3,159,Albania,2002,male,55-74 years,13,198130,6.56,Albania2002,"4,435,078,648",1573,Silent
4,160,Albania,2002,male,15-24 years,15,263067,5.70,Albania2002,"4,435,078,648",1573,Millenials
5,161,Albania,2002,female,15-24 years,14,275970,5.07,Albania2002,"4,435,078,648",1573,Millenials
6,162,Albania,2002,female,35-54 years,15,375113,4.00,Albania2002,"4,435,078,648",1573,Boomers
7,163,Albania,2002,female,25-34 years,7,223685,3.13,Albania2002,"4,435,078,648",1573,Generation X
8,164,Albania,2002,female,75+ years,1,47407,2.11,Albania2002,"4,435,078,648",1573,Silent
9,165,Albania,2002,female,55-74 years,4,191712,2.09,Albania2002,"4,435,078,648",1573,Silent


In [210]:
suicide.tail(22)

,index,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),generation
14058,27798,Uzbekistan,2013,female,15-24 years,323,3039740,10.63,Uzbekistan2013,"57,690,453,461",2150,Millenials
14059,27799,Uzbekistan,2013,male,15-24 years,320,3171202,10.09,Uzbekistan2013,"57,690,453,461",2150,Millenials
14060,27800,Uzbekistan,2013,male,55-74 years,119,1202790,9.89,Uzbekistan2013,"57,690,453,461",2150,Boomers
14061,27801,Uzbekistan,2013,male,75+ years,13,221002,5.88,Uzbekistan2013,"57,690,453,461",2150,Silent
14062,27802,Uzbekistan,2013,female,25-34 years,146,2647820,5.51,Uzbekistan2013,"57,690,453,461",2150,Millenials
14063,27803,Uzbekistan,2013,female,35-54 years,99,3547895,2.79,Uzbekistan2013,"57,690,453,461",2150,Generation X
14064,27804,Uzbekistan,2013,female,75+ years,8,345180,2.32,Uzbekistan2013,"57,690,453,461",2150,Silent
14065,27805,Uzbekistan,2013,male,5-14 years,61,2720938,2.24,Uzbekistan2013,"57,690,453,461",2150,Generation Z
14066,27806,Uzbekistan,2013,female,55-74 years,21,1356298,1.55,Uzbekistan2013,"57,690,453,461",2150,Boomers
14067,27807,Uzbekistan,2013,female,5-14 years,31,2595000,1.19,Uzbekistan2013,"57,690,453,461",2150,Generation Z


In [0]:
# There are supposed to be 12 age categories per country per year

suicidesAgeCategoriesByCountryYear = pd.DataFrame( suicide['year'].groupby(suicide['country-year']).count() ).reset_index()
suicidesAgeCategoriesByCountryYear.columns = ['country-year', 'age-category-count']

In [212]:
suicidesAgeCategoriesByCountryYear.loc[ suicidesAgeCategoriesByCountryYear['age-category-count'] != 12 ]

,country-year,age-category-count
47,Armenia2016,10
85,Austria2016,10
261,Croatia2016,10
288,Cyprus2016,10
303,Czech Republic2016,10
449,Grenada2016,10
490,Hungary2016,10
505,Iceland2016,10
638,Lithuania2016,10
689,Mauritius2016,10


In [213]:
# Remove all suicide data for 2016 since it is incomplete
suicide = suicide.loc[ suicide.year < 2016 ].reset_index()

suicide.head(10)

,level_0,index,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),generation
0,0,156,Albania,2002,male,75+ years,4,31007,12.90,Albania2002,"4,435,078,648",1573,Silent
1,1,157,Albania,2002,male,25-34 years,23,206286,11.15,Albania2002,"4,435,078,648",1573,Generation X
2,2,158,Albania,2002,male,35-54 years,35,382139,9.16,Albania2002,"4,435,078,648",1573,Boomers
3,3,159,Albania,2002,male,55-74 years,13,198130,6.56,Albania2002,"4,435,078,648",1573,Silent
4,4,160,Albania,2002,male,15-24 years,15,263067,5.70,Albania2002,"4,435,078,648",1573,Millenials
5,5,161,Albania,2002,female,15-24 years,14,275970,5.07,Albania2002,"4,435,078,648",1573,Millenials
6,6,162,Albania,2002,female,35-54 years,15,375113,4.00,Albania2002,"4,435,078,648",1573,Boomers
7,7,163,Albania,2002,female,25-34 years,7,223685,3.13,Albania2002,"4,435,078,648",1573,Generation X
8,8,164,Albania,2002,female,75+ years,1,47407,2.11,Albania2002,"4,435,078,648",1573,Silent
9,9,165,Albania,2002,female,55-74 years,4,191712,2.09,Albania2002,"4,435,078,648",1573,Silent


In [0]:
# Check there is data for each country per year

yearsPerCountry = suicide['year'].groupby(suicide['country']).count().reset_index()
yearsPerCountry.columns = ['country', 'year-count']

In [0]:
#suicide[ suicide.country == 'Australia' ].groupby( suicide['year'] ).count()

In [0]:
# Only keep data for countries with the full 14 years of 12 age categories (168 records)
suicide = suicide[ suicide.country.isin( yearsPerCountry[ yearsPerCountry['year-count'] == 168 ].country.get_values() ) ]

In [217]:
suicide.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8736 entries, 252 to 13643
Data columns (total 13 columns):
level_0               8736 non-null int64
index                 8736 non-null int64
country               8736 non-null object
year                  8736 non-null int64
sex                   8736 non-null object
age                   8736 non-null object
suicides_no           8736 non-null int64
population            8736 non-null int64
suicides/100k pop     8736 non-null float64
country-year          8736 non-null object
 gdp_for_year ($)     8736 non-null object
gdp_per_capita ($)    8736 non-null int64
generation            8736 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 955.5+ KB


In [269]:
suicideCountByYear = suicide.suicides_no.groupby(suicide.year).sum().reset_index()
suicideCountByYear

,year,suicides_no
0,2002,209833
1,2003,209727
2,2004,206618
3,2005,203087
4,2006,197136
5,2007,197410
6,2008,198853
7,2009,203506
8,2010,198781
9,2011,198358


In [339]:
suicideCountByYear['norm_suicides_no'] = preprocessing.normalize([suicideCountByYear.suicides_no.get_values()], norm='l1')[0]
suicideCountByYear

,year,suicides_no,norm_suicides_no
0,2002,209833,0.075051
1,2003,209727,0.075013
2,2004,206618,0.073901
3,2005,203087,0.072638
4,2006,197136,0.070510
5,2007,197410,0.070608
6,2008,198853,0.071124
7,2009,203506,0.072788
8,2010,198781,0.071098
9,2011,198358,0.070947


In [284]:
output_notebook()

p = figure(title='Total Number of Suicides by Year (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='Number of suicides'
          )

p.line(x='year', y='suicides_no', source=suicideCountByYear, line_width=2, line_color='black')
p.circle(x='year', y='suicides_no', size=10, source=suicideCountByYear, color='#CC0000')


p.add_tools(HoverTool(tooltips='@suicides_no'))

p.toolbar.autohide = True
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

show(p)

Loading BokehJS ...

In [218]:
suicideCountByCountry = suicide.suicides_no.groupby(suicide.country).sum().reset_index()
suicideCountByCountry


,country,suicides_no
0,Argentina,43021
1,Austria,18641
2,Belgium,27775
3,Belize,236
4,Brazil,129916
5,Chile,25787
6,Colombia,30617
7,Croatia,11045
8,Cuba,20599
9,Czech Republic,21226


In [266]:
output_notebook()

p = figure(title='Total Number of Suicides by Country (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Number of suicides',
           y_axis_label='Country',
           y_range=suicideCountByCountryYear.country.unique()
          )

p.hbar(left=0, right='suicides_no',
       y='country',
       source=suicideCountByCountry,
       height=0.5,
       color='black'
      )

p.add_tools(HoverTool(tooltips='@suicides_no'))

p.toolbar.autohide = True
p.xaxis.formatter = NumeralTickFormatter(format="0,0")

show(p)

Loading BokehJS ...

In [220]:
suicideCountByCountryYear = suicide.suicides_no.groupby([suicide.country, suicide.year]).sum().reset_index()
suicideCountByCountryYear


,country,year,suicides_no
0,Argentina,2002,3162
1,Argentina,2003,3289
2,Argentina,2004,3121
3,Argentina,2005,3058
4,Argentina,2006,3108
5,Argentina,2007,2946
6,Argentina,2008,3059
7,Argentina,2009,2884
8,Argentina,2010,2943
9,Argentina,2011,2912


In [0]:
suicideCountByCountryYear['sizes'] = preprocessing.normalize([suicideCountByCountryYear.suicides_no.get_values()])[0] * 100


In [299]:
suicideCountByCountryYear.reset_index()

,index,country,year,suicides_no,sizes
0,0,Argentina,2002,3162,1.288313
1,1,Argentina,2003,3289,1.340058
2,2,Argentina,2004,3121,1.271608
3,3,Argentina,2005,3058,1.245940
4,4,Argentina,2006,3108,1.266312
5,5,Argentina,2007,2946,1.200307
6,6,Argentina,2008,3059,1.246347
7,7,Argentina,2009,2884,1.175046
8,8,Argentina,2010,2943,1.199085
9,9,Argentina,2011,2912,1.186454


In [300]:
# Plot count of suicides by year by country

output_notebook()

p = figure(title='Number of Suicides by Country by Year (2002 - 2015)', 
           plot_width=1000, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='Country',
           y_range=suicideCountByCountryYear.country.unique()
          )

leg1 = p.circle('year', 'country', size='sizes', source=suicideCountByCountryYear)

p.add_tools(HoverTool(tooltips='@suicides_no'))

p.toolbar.autohide = True
p.xaxis.ticker = SingleIntervalTicker(interval=1)

legend = Legend(items=[('Number of suicides per year', [leg1])], location=(10, 400))
p.add_layout(legend, 'right')

show(p)

Loading BokehJS ...

## Solar Flare Data

In [231]:
!wget -N "https://raw.githubusercontent.com/channeladam/MOOCs/master/SchoolOfAI/DataLit/Week03/Data/hessi.solar.flare.2002to2016.csv"


--2019-02-17 13:57:56--  https://raw.githubusercontent.com/channeladam/MOOCs/master/SchoolOfAI/DataLit/Week03/Data/hessi.solar.flare.2002to2016.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11003842 (10M) [text/plain]
Saving to: ‘hessi.solar.flare.2002to2016.csv’

hessi.solar.flare.2 100%[===================>]  10.49M  --.-KB/s    in 0.08s   

Last-modified header missing -- time-stamps turned off.
2019-02-17 13:57:57 (126 MB/s) - ‘hessi.solar.flare.2002to2016.csv’ saved [11003842/11003842]



In [301]:
flares = pd.read_csv('/content/hessi.solar.flare.2002to2016.csv')
flares.head(10)

,flare,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5
0,2021213,2002-02-12,21:29:56,21:33:38,21:41:48,712,136,167304,12-25,592,-358,692,0,A1,P1,NaN,NaN,NaN
1,2021228,2002-02-12,21:44:08,21:45:06,21:48:56,288,7,9504,6-12,604,-341,694,9811,A1,P1,PE,Q1,NaN
2,2021332,2002-02-13,00:53:24,00:54:54,00:57:00,216,15,11448,6-12,-310,375,487,9825,A1,P1,NaN,NaN,NaN
3,2021308,2002-02-13,04:22:52,04:23:50,04:26:56,244,20,17400,12-25,-277,378,469,9822,A1,P1,NaN,NaN,NaN
4,2021310,2002-02-13,07:03:52,07:05:14,07:07:48,236,336,313392,25-50,-272,390,476,9825,A1,GS,P1,PE,Q2
5,2021353,2002-02-13,07:07:48,07:09:14,07:20:56,788,272,524304,12-25,-271,389,474,9825,A1,P1,NaN,NaN,NaN
6,2021354,2002-02-13,07:20:56,07:22:42,07:30:04,548,28,52488,6-12,-267,390,473,9825,A1,P1,NaN,NaN,NaN
7,2021312,2002-02-13,08:53:20,08:55:18,09:05:08,708,92,125352,25-50,-362,309,476,0,A1,P1,NaN,NaN,NaN
8,2021339,2002-02-13,10:02:56,10:04:42,10:04:44,108,26,10368,6-12,-235,390,455,9822,A1,P1,PE,Q2,SE
9,2021313,2002-02-13,12:29:32,12:30:58,12:33:24,232,26,16920,12-25,-905,-375,980,0,A1,P1,NaN,NaN,NaN


In [285]:
flares.tail(10)

,flare,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5
113932,16121601,2016-12-16,10:36:00,10:37:14,10:38:20,140,112,10312,3-6,0,0,0,0,A0,DF,NS,PE,Q2
113933,16122101,2016-12-21,14:20:04,14:21:38,14:26:28,384,92,37184,3-6,0,0,0,0,A0,DF,DR,NS,PE Q3
113934,16122102,2016-12-21,16:16:40,16:16:42,16:16:48,8,24,368,6-12,914,-84,918,0,A0,DF,P1,PE,Q2
113935,16122601,2016-12-26,18:48:16,18:50:22,18:57:48,572,208,109744,3-6,0,0,0,0,A0,GE,NS,PE,Q2
113936,16122602,2016-12-26,21:59:56,22:02:10,22:09:20,564,288,126976,3-6,0,0,0,0,A0,DR,NS,PE,Q2
113937,16122603,2016-12-26,22:09:20,22:17:02,22:17:24,484,152,75328,3-6,0,0,0,0,A0,GE,NS,PE,Q2
113938,16122604,2016-12-26,23:25:32,23:28:18,23:43:48,1096,304,142048,3-6,0,0,0,0,A0,DR,NS,PE,Q2
113939,16122701,2016-12-27,08:33:08,08:39:06,08:39:40,392,80,32344,3-6,0,0,0,0,A0,DF,NS,PE,Q3 SE
113940,16122702,2016-12-27,12:21:24,12:21:30,12:30:04,520,24,17336,6-12,326,326,461,0,A0,DF,DR,P1,PE Q3
113941,16122703,2016-12-27,12:35:40,12:37:14,12:39:00,200,48,11056,6-12,765,198,791,2621,A0,DF,P1,PE,Q2


In [268]:
flares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113942 entries, 0 to 113941
Data columns (total 18 columns):
flare               113942 non-null int64
start.date          113942 non-null object
start.time          113942 non-null object
peak                113942 non-null object
end                 113942 non-null object
duration.s          113942 non-null int64
peak.c/s            113942 non-null int64
total.counts        113942 non-null int64
energy.kev          113942 non-null object
x.pos.asec          113942 non-null int64
y.pos.asec          113942 non-null int64
radial              113942 non-null int64
active.region.ar    113942 non-null int64
flag.1              113942 non-null object
flag.2              113942 non-null object
flag.3              94035 non-null object
flag.4              93740 non-null object
flag.5              52991 non-null object
dtypes: int64(8), object(10)
memory usage: 15.6+ MB


In [308]:
# Remove all flare data for 2016 since suicide data for 2016 has been removed

flares['year'] = flares['start.date'].str[:4].astype(int)

flares = flares[ flares.year != 2016 ].reset_index()
flares

,index,flare,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5,year
0,0,2021213,2002-02-12,21:29:56,21:33:38,21:41:48,712,136,167304,12-25,592,-358,692,0,A1,P1,NaN,NaN,NaN,2002
1,1,2021228,2002-02-12,21:44:08,21:45:06,21:48:56,288,7,9504,6-12,604,-341,694,9811,A1,P1,PE,Q1,NaN,2002
2,2,2021332,2002-02-13,00:53:24,00:54:54,00:57:00,216,15,11448,6-12,-310,375,487,9825,A1,P1,NaN,NaN,NaN,2002
3,3,2021308,2002-02-13,04:22:52,04:23:50,04:26:56,244,20,17400,12-25,-277,378,469,9822,A1,P1,NaN,NaN,NaN,2002
4,4,2021310,2002-02-13,07:03:52,07:05:14,07:07:48,236,336,313392,25-50,-272,390,476,9825,A1,GS,P1,PE,Q2,2002
5,5,2021353,2002-02-13,07:07:48,07:09:14,07:20:56,788,272,524304,12-25,-271,389,474,9825,A1,P1,NaN,NaN,NaN,2002
6,6,2021354,2002-02-13,07:20:56,07:22:42,07:30:04,548,28,52488,6-12,-267,390,473,9825,A1,P1,NaN,NaN,NaN,2002
7,7,2021312,2002-02-13,08:53:20,08:55:18,09:05:08,708,92,125352,25-50,-362,309,476,0,A1,P1,NaN,NaN,NaN,2002
8,8,2021339,2002-02-13,10:02:56,10:04:42,10:04:44,108,26,10368,6-12,-235,390,455,9822,A1,P1,PE,Q2,SE,2002
9,9,2021313,2002-02-13,12:29:32,12:30:58,12:33:24,232,26,16920,12-25,-905,-375,980,0,A1,P1,NaN,NaN,NaN,2002


In [313]:
flareEnergyCategories = flares['energy.kev'].unique()
flareEnergyCategories

array(['12-25', '6-12', '25-50', '3-6', '50-100', '100-300', '300-800',
       '7000-20000', '800-7000'], dtype=object)

In [321]:
flaresByYear = flares.groupby('year').agg({'total.counts': ['count', 'sum', 'mean', 'std', 'var', 'min', 'max']}).reset_index()
flaresByYear.columns = ['year', 'count', 'sum', 'mean', 'std', 'var', 'min', 'max']
flaresByYear

,year,count,sum,mean,std,var,min,max
0,2002,9582,6304147032,657915.574202,4.141575e+06,1.715265e+13,52,237456224
1,2003,14700,7155290046,486754.424898,5.624038e+06,3.162981e+13,44,435550080
2,2004,10753,4333914110,403042.324003,1.722305e+06,2.966333e+12,37,75888848
3,2005,7824,4146666084,529993.108896,4.841726e+06,2.344231e+13,23,318884832
4,2006,3345,1599217227,478091.846637,5.025662e+06,2.525728e+13,92,248619072
5,2007,2053,416101885,202679.924501,5.913927e+05,3.497453e+11,50,9657240
6,2008,556,67067538,120625.068345,2.954700e+05,8.730255e+10,4608,4173583
7,2009,765,97020109,126823.671895,3.146668e+05,9.901518e+10,470,4830192
8,2010,2605,534525972,205192.311708,6.155934e+05,3.789553e+11,126,12243648
9,2011,11842,3584039093,302654.880341,1.557691e+06,2.426400e+12,66,97891880


In [340]:
flaresByYear['norm_count'] = preprocessing.normalize([flaresByYear['count'].get_values()], norm='l1')[0]
flaresByYear['norm_sum'] = preprocessing.normalize([flaresByYear['sum'].get_values()], norm='l1')[0]
flaresByYear

,year,count,sum,mean,std,var,min,max,norm_count,norm_sum
0,2002,9582,6304147032,657915.574202,4.141575e+06,1.715265e+13,52,237456224,0.086052,0.148381
1,2003,14700,7155290046,486754.424898,5.624038e+06,3.162981e+13,44,435550080,0.132015,0.168414
2,2004,10753,4333914110,403042.324003,1.722305e+06,2.966333e+12,37,75888848,0.096569,0.102008
3,2005,7824,4146666084,529993.108896,4.841726e+06,2.344231e+13,23,318884832,0.070264,0.097600
4,2006,3345,1599217227,478091.846637,5.025662e+06,2.525728e+13,92,248619072,0.030040,0.037641
5,2007,2053,416101885,202679.924501,5.913927e+05,3.497453e+11,50,9657240,0.018437,0.009794
6,2008,556,67067538,120625.068345,2.954700e+05,8.730255e+10,4608,4173583,0.004993,0.001579
7,2009,765,97020109,126823.671895,3.146668e+05,9.901518e+10,470,4830192,0.006870,0.002284
8,2010,2605,534525972,205192.311708,6.155934e+05,3.789553e+11,126,12243648,0.023394,0.012581
9,2011,11842,3584039093,302654.880341,1.557691e+06,2.426400e+12,66,97891880,0.106348,0.084358


In [330]:
output_notebook()

p = figure(title='Number of Solar Flare Events by Year (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='Number of Flare Events'
          )

p.line(x='year', y='count', source=flaresByYear, line_width=4, line_color='gold')
p.circle(x='year', y='count', size=10, source=flaresByYear, color='green')


p.add_tools(HoverTool(tooltips='@count'))

p.toolbar.autohide = True
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

show(p)

Loading BokehJS ...

In [331]:
output_notebook()

p = figure(title='Integrated keV Energy Count by Year (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='Integrated keV Energy Count'
          )

p.line(x='year', y='sum', source=flaresByYear, line_width=4, line_color='gold')
p.circle(x='year', y='sum', size=10, source=flaresByYear, color='green')


p.add_tools(HoverTool(tooltips='@sum'))

p.toolbar.autohide = True
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

show(p)

Loading BokehJS ...

# Writeup


## Aim
Is there a correlation between solar flare activity and rates of suicide?

## Data Assumptions
Data used in this analysis is from the year 2002 until 2015 inclusive.

Data does NOT include all countries in the world.
Some countries were excluded from the dataset due to having incomplete data.

Countries included in the sample are as below.

In [355]:
for c in suicideCountByCountryYear.country.unique():
  print(c)

Argentina
Austria
Belgium
Belize
Brazil
Chile
Colombia
Croatia
Cuba
Czech Republic
Denmark
Ecuador
Estonia
Finland
Germany
Greece
Grenada
Guatemala
Hungary
Iceland
Israel
Italy
Japan
Kazakhstan
Kyrgyzstan
Latvia
Lithuania
Luxembourg
Malta
Mauritius
Mexico
Netherlands
Norway
Panama
Poland
Puerto Rico
Republic of Korea
Romania
Russian Federation
Saint Vincent and Grenadines
Serbia
Seychelles
Singapore
Slovenia
South Africa
Spain
Sweden
Switzerland
Thailand
Turkmenistan
United Kingdom
United States


## Let's Investigate Suicide by Country

In [351]:
output_notebook()

p = figure(title='Total Number of Suicides by Country (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Number of suicides',
           y_axis_label='Country',
           y_range=suicideCountByCountryYear.country.unique()
          )

p.hbar(left=0, right='suicides_no',
       y='country',
       source=suicideCountByCountry,
       height=0.5,
       color='black'
      )

p.add_tools(HoverTool(tooltips='@suicides_no'))

p.toolbar.autohide = True
p.xaxis.formatter = NumeralTickFormatter(format="0,0")

show(p)

Loading BokehJS ...

From the sample above, the following countries lead the world in suicide for the period between 2002 and 2015:
- Russia
- United States
- Japan



## Let's Investigate Suicide by Country, by Year.


In [352]:
# Plot count of suicides by year by country

output_notebook()

p = figure(title='Number of Suicides by Country by Year (2002 - 2015)', 
           plot_width=1000, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='Country',
           y_range=suicideCountByCountryYear.country.unique()
          )

leg1 = p.circle('year', 'country', size='sizes', source=suicideCountByCountryYear)

p.add_tools(HoverTool(tooltips='@suicides_no'))

p.toolbar.autohide = True
p.xaxis.ticker = SingleIntervalTicker(interval=1)

legend = Legend(items=[('Number of suicides per year', [leg1])], location=(10, 400))
p.add_layout(legend, 'right')

show(p)

Loading BokehJS ...

Of interest:

* The suicide rate in the United States has increased steadily from 31645 in 2002 to  44189 in 2015.

* The suicide rate in Russia has decreased steadily from 55024 in 2002 to 25432 in 2015.

* From 2010, Japan has decreased their suicide rate 23092 in 2015, when it started at 29715 in 2002 and peaked in 2007 at 30665

## Let's Investigate the Total Suicide Counts
For all countries in the sample, combined.





In [350]:
output_notebook()

p = figure(title='Total Number of Suicides by Year (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='Number of suicides'
          )

p.line(x='year', y='suicides_no', source=suicideCountByYear, line_width=2, line_color='black')
p.circle(x='year', y='suicides_no', size=10, source=suicideCountByYear, color='#CC0000')


p.add_tools(HoverTool(tooltips='@suicides_no'))

p.toolbar.autohide = True
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

show(p)

Loading BokehJS ...

Of interest:

* Globally, there was an increase in suicide from 2007 until 2009 inclusive.

* Total suicide counts have decreased from 209,833 in 2002 to 190,189 in 2015.

# Let's Investigate the Solar Flare Activity by Year


In [356]:
output_notebook()

p = figure(title='Number of Solar Flare Events by Year (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='Number of Flare Events'
          )

p.line(x='year', y='count', source=flaresByYear, line_width=4, line_color='gold')
p.circle(x='year', y='count', size=10, source=flaresByYear, color='green')


p.add_tools(HoverTool(tooltips='@count'))

p.toolbar.autohide = True
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

show(p)

Loading BokehJS ...

Of interest, the number of solar flare events was:
* Decreasing significantly from 14,700 in 2003 to 556 in 2008 
* Increasing from 765 in 2009 to 11,842 in 2011

## Suicide & Solar Flares

The graph below shows an overlay of the normalised amount of solar flare energy versus the normalised count of suicides in the world.

In [349]:
output_notebook()

p = figure(title='Suicides VS Solar Flare Energy by Year (2002 - 2015)', 
           plot_width=800, plot_height=800, toolbar_location="left",
           x_axis_label='Year',
           y_axis_label='L1 Normalised Value'
          )

p.line(x='year', y='norm_suicides_no', source=suicideCountByYear, line_width=2, line_color='black')
p.circle(x='year', y='norm_suicides_no', size=10, source=suicideCountByYear, color='#CC0000')

p.line(x='year', y='norm_sum', source=flaresByYear, line_width=2, line_color='gold')
p.circle(x='year', y='norm_sum', size=10, source=flaresByYear, color='green')

p.toolbar.autohide = True


show(p)

Loading BokehJS ...

## Conclusion

There is no visually apparent correlation between the amount of solar flare energy expelled from the Sun and total counts of suicide in the world, between 2002 and 2015.

## Further Work
If I had time, I would:
* enhance the graph of suicide by country, by year to be animated, showing one year at a time.
* properly type up the null hypothesis and perform a T-test for a purely statistical (rather than visual) insignificance.
* show an area chart of the solar flare energy categories over time